<img src='https://www.di.uniroma1.it/sites/all/themes/sapienza_bootstrap/logo.png' width="200"/>  

# Part_1_1_Syntax_and_CKY  

In Natural Language Processing (`NLP`), syntax helps analyze and generate human language. **Context-free grammars (`CFGs`)** describe sentence structures and are used in tasks like parsing and translation.

The **Cocke–Younger–Kasami (`CKY`) algorithm** is a key method for processing `CFGs`, efficiently checking if a sentence matches a grammar and building its parse tree. This notebook covers `CFGs` basics and explains the `CKY` algorithm.

### **Objectives:**  
In this notebook, Parham introduces **context-free grammars** as a foundation for syntactic parsing in `NLP` and demonstrates their role in defining sentence structure. The tutorial will then cover the `CKY` algorithm, explaining its theory and practical implementation with hands-on examples.

### **References:**  
- [https://web.stanford.edu/class/archive/cs/cs103/cs103.1164/lectures/18/Small18.pdf](https://web.stanford.edu/class/archive/cs/cs103/cs103.1164/lectures/18/Small18.pdf)  
- [https://www.nltk.org/api/nltk.html#nltk.tree.Tree](https://www.nltk.org/api/nltk.html#nltk.tree.Tree) 
- [https://github.com/NLie2/CYK-algorithm-for-CFG](https://github.com/NLie2/CYK-algorithm-for-CFG)  
- [https://github.com/Huntersxsx/CKY-Parser](https://github.com/Huntersxsx/CKY-Parser)

### **Contributors:**  
- Parham Membari  
    - <img src="https://upload.wikimedia.org/wikipedia/commons/7/7e/Gmail_icon_%282020%29.svg" alt="Logo" width="20" height="20"> **Email**: p.membari96@gmail.com  
    - <img src="https://www.iconsdb.com/icons/preview/red/linkedin-6-xxl.png" alt="Logo" width="20" height="20"> **LinkedIn**: [LinkedIn](https://www.linkedin.com/in/p-mem/)  
    - <img src="https://upload.wikimedia.org/wikipedia/commons/a/ae/Github-desktop-logo-symbol.svg" alt="Logo" width="20" height="20"> **GitHub**: [GitHub](https://github.com/parham075)  
    - <img src="https://upload.wikimedia.org/wikipedia/commons/e/ec/Medium_logo_Monogram.svg" alt="Logo" width="20" height="20"> **Medium**: [Medium](https://medium.com/@p.membari96)  

**Table of Contents:**  
1. Import Libraries  
2. Introduction to Syntax in NLP  
3. Context-Free Grammars (`CFGs`)   
4. The Cocke–Younger–Kasami (`CKY`) Algorithm    
5. Conclusion  

## 1. Import Libraries 

In [1]:
import nltk
from nltk import CFG
from nltk.parse.chart import ChartParser

## 2. Introduction to Syntax in NLP

Syntax is a key aspect of linguistics that focuses on the arrangement of words and phrases to form valid sentences. In Natural Language Processing (`NLP`), understanding syntax is crucial for analyzing the grammatical structure of text, enabling machines to interpret and generate language more effectively.  

Syntactic analysis involves identifying the relation of words to each other in a sentence, demonstrating the underlying structure. For example, in the sentence:  

``` The dog chased the cat. ``` 

Syntax helps determine relationships like *"the dog"* as the subject and *"chased the cat"* as the predicate.  

In the following sections, we focus on **context-free grammars (`CFGs`)**, a key formalism for syntactic analysis, and the **Cocke–Younger–Kasami (`CKY`) algorithm**, a method for parsing sentences using CFGs.  

### **3. Context-Free Grammars (`CFGs`)**  

Parham explores two foundational models for regular languages:  
1. **[Regular Expressions](notebooks/Part_1_3_Regular_Expressions.ipynb)**:  
   These describe the strings in a language, match strings belonging to the language, and define the general structure of all valid strings.  

2. **[Finite Automata](notebooks/Part_1_4_REs_and_finite_state_automata.ipynb)**:  
   These accept strings in the language, recognize valid strings, and perform computations to determine membership of specific strings.  

While these models are powerful for defining **regular languages**, more complex languages require additional formalism.  

#### **What is a Context-Free Grammar?**  
A **context-free grammar (`CFG`)** is a formalism for defining languages beyond the capabilities of regular expressions and finite automata. Unlike regular languages, `CFGs` focus on generating valid strings in a language through a structured process, often producing hierarchical structures like parse trees. The primary goal of `CFGs` is to provide a procedure for listing all strings in a language, making them essential for syntactic analysis in `NLP`. 

#### **Explain CFG with example:**
A **context-free grammar (CFG)** provides a systematic way to generate all valid strings in a language. Unlike regular expressions and finite automata, which are limited to regular languages, `CFGs` can handle more complex languages with hierarchical structures, such as arithmetic expressions or programming languages.  

**Arithmetic Expressions**  
Suppose we want to describe all valid arithmetic expressions using addition, subtraction, multiplication, and division. Here’s a possible CFG for this:  

```  
E → int  
E → E Op E  
E → (E)  
Op → +  
Op → -  
Op → *  
Op → /  
```  

This grammar defines valid expressions like `int + int`, `int * (int - int)`, or `(int + int) / int`. Let's look at a derivation step-by-step:  

**Derivation Example 1**  
To derive the expression `int * (int + int)`:  
1. Start with `E`.  
2. Apply `E → E Op E` → `E * E`.  
3. Replace the first `E` with `int` → `int * E`.  
4. Apply `E → (E)` to the second `E` → `int * (E)`.  
5. Apply `E → E Op E` → `int * (E + E)`.  
6. Replace both `E`s with `int` → `int * (int + int)`.  

**Derivation Example 2**  
To derive the expression `int / int`:  
1. Start with `E`.  
2. Apply `E → E Op E` → `E / E`.  
3. Replace both `E`s with `int` → `int / int`.  

**Formal Definition of a CFG**  
A **context-free grammar** is defined as a collection of four components:  
1. **Nonterminal Symbols**: Variables representing intermediate structures (e.g., `E`, `Op`).  
2. **Terminal Symbols**: The alphabet of the language (e.g., `int`, `+`, `-`).  
3. **Production Rules**: Specify how nonterminals can be replaced by a string of terminals and/or other nonterminals (e.g., `E → int`, `E → E Op E`).  
4. **Start Symbol**: A special nonterminal (e.g., `E`) from which derivations begin.  

**Notation and Shorthand**  
- **Nonterminals**: Represented with capital letters (e.g., `E`, `Op`).  
- **Terminals**: Represented with lowercase or symbolic characters (e.g., `int`, `+`).  
- **Derivations**: A sequence of steps replacing nonterminals by the right-hand side of a production.  

For example:  
```
E → int | E Op E | (E)  
Op → + | - | * | /  
```  
This shorthand combines rules for readability.  

**Language of a CFG**  
If `G` is a CFG with an alphabet `Σ` and start symbol `S`, the **language** of `G`, denoted as `ℒ(G)`, is the set of all strings derivable from `S` using the grammar's rules.  

Formally:  
```
ℒ(G) = { ω ∈ Σ* | S ⇒* ω }  
```
Where `ω` is a string of terminals, and `⇒*` means "derivable in zero or more steps."  

#### **Context-Free Languages (CFLs)**  
A language `L` is called a **context-free language (CFL)** if there exists a CFG `G` such that `L = ℒ(G)`. 


In [4]:
import nltk
from nltk import CFG
from nltk.parse.chart import ChartParser

# Define a context-free grammar (CFG) correctly
cfg_grammar = CFG.fromstring("""
  E -> 'int' | E Op E | '(' E ')'
  Op -> '+' | '-' | '*' | '/'
""")

# Print the grammar rules
print("Context-Free Grammar Rules:")
print(cfg_grammar)

# Example arithmetic expression to parse
expression = "int * ( int + int )"

# Tokenize the expression
tokens = expression.split()
print("\nTokens:", tokens)

# Create a chart parser with the defined CFG
parser = ChartParser(cfg_grammar)

# Parse the tokens and print all valid parse trees
print("\nParse Trees:")
for tree in parser.parse(tokens):
    print(tree)
    tree.pretty_print()


Context-Free Grammar Rules:
Grammar with 7 productions (start state = E)
    E -> 'int'
    E -> E Op E
    E -> '(' E ')'
    Op -> '+'
    Op -> '-'
    Op -> '*'
    Op -> '/'

Tokens: ['int', '*', '(', 'int', '+', 'int', ')']

Parse Trees:
(E (E int) (Op *) (E ( (E (E int) (Op +) (E int)) )))
             E             
  ___________|___           
 |   |           E         
 |   |    _______|___       
 |   |   |   |       E     
 |   |   |   |    ___|___   
 E   Op  |   |   E   Op  E 
 |   |   |   |   |   |   |  
int  *   (   )  int  +  int



## 4. The Cocke–Younger–Kasami (`CKY`) Algorithm  

## 5. Conclusion 